In [ ]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
#!pip install geopy
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
!pip install folium
import folium 

  Using cached https://files.pythonhosted.org/packages/0c/67/915668d0e286caa21a1da82a85ffe3d20528ec7212777b43ccd027d94023/geopy-2.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/62/26ec95a98ba64299163199e95ad1b0e34ad3f4e176e221c40245f211e425/geographiclib-1.50-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/83/e8cb37afc2f016a1cf4caab8d22caf7fe4156c4c15230d8abc9c83547e0c/folium-0.12.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/1f/570b0615c452265d57e4114e633231d6cd9b9d275256778a675681e4f711/branca-0.4.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/

In [ ]:
url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=995657573').text
soup = BeautifulSoup(url,'lxml')

In [ ]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

Dropping Borough which are not assigned.

In [ ]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

df.head(20)

Grouping the records based on Postcode.

In [ ]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df

In [ ]:
df.shape

In [ ]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [ ]:
df_pos = pd.merge(df, df_geo, on=['Postcode'], how='inner')
df_tor = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_tor.head()

# Q3: Drawing inspiration from the previous lab where we cluster the neighbourhood of NYC, We cluster Toronto based on the similarities of the venues categories using Kmeans clustering and Foursquare API.

In [ ]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Postcode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The coordinates of Toronto are 43.6534817, -79.3839347.


Initializing Foursquare API credentials

In [ ]:
CLIENT_ID = 'TMELJWN2FBOQOEQYIKG5ESBXC5CJJN3UYZB3QON3YF2OKDG3' # your Foursquare ID
CLIENT_SECRET = 'DG2Z5VCB2BM4NQYQL0AEWLL1X2Q0OBPDJB0YGOGKVP4CDMMT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Next, we create a function to get all the venue categories in Toronto

In [ ]:
# The following function retrieves the venues given the names and coordinates and stores it into dataframe.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Collecting the venues in Toronto for each Neighbourhood

In [ ]:
df_tor_venues = getNearbyVenues(names=df_tor['Neighbourhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

In [ ]:
print(df_tor_venues.shape)
df_tor_venues.head()

So we have 1346 records and 7 columns. Checking sample data

In [ ]:
df_tor_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(df_tor_venues['Venue Category'].unique())))

# One Hot encoding the venue Categories

In [ ]:
# one hot encoding
onehot = pd.get_dummies(df_tor_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
onehot['Neighbourhood'] = df_tor_venues['Neighbourhood'] 

onehot.head()

Adding the neighbourhood to the encoded dataframe

Now we will group the Neighbourhoods, calculate the mean venue categories in each Neighbourhood

In [ ]:
gd = onehot.groupby('Neighbourhood').mean().reset_index()
gd

In [ ]:
gd.shape

In [ ]:
num_top_venues = 5

for hood in gd['Neighbourhood']:
    print("----"+hood+"----")
    temp = gd[gd['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = gd['Neighbourhood']

for ind in np.arange(gd.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(gd.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Let's make a function to get the top most common venue categories

Let's make the model to cluster our Neighbourhoods

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
k_num_clusters = 5

tor_gd_clustering = gd.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(tor_gd_clustering)
kmeans

Checking the labelling of our model

In [ ]:
kmeans.labels_[0:100]

Let's add the clustering Label column to the top 10 common venue categories

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
neighborhoods_venues_sorted

Join df_tor_mer with df_tor on neighbourhood to add latitude & longitude for each neighborhood to prepare it for plotting

In [ ]:
df_tor_mer = df_tor
df_tor_mer= df_tor_mer.merge(neighborhoods_venues_sorted, left_on='Neighbourhood', right_on='Neighbourhood')

df_tor_mer.head()

Drop all the nan values to prevent data skew

In [ ]:
df_tor_mer = df_tor_mer.dropna(subset=['Cluster Labels'])


Plotting the clusters on the map

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_mer['Latitude'], df_tor_mer['Longitude'], df_tor_mer['Neighbourhood'], df_tor_mer['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

Let's verify each of our clusters

Cluster 1

In [ ]:
df_tor_mer.loc[df_tor_mer['Cluster Labels'] == 0, df_tor_mer.columns[[1] + list(range(5, df_tor_mer.shape[1]))]]

Cluster 2

In [ ]:
df_tor_mer.loc[df_tor_mer['Cluster Labels'] == 1, df_tor_mer.columns[[1] + list(range(5, df_tor_mer.shape[1]))]]

Cluster 3

In [ ]:
df_tor_mer.loc[df_tor_mer['Cluster Labels'] == 2, df_tor_mer.columns[[1] + list(range(5, df_tor_mer.shape[1]))]]

Cluster 4

In [ ]:
df_tor_mer.loc[df_tor_mer['Cluster Labels'] == 3, df_tor_mer.columns[[1] + list(range(5, df_tor_mer.shape[1]))]]

Cluster 5

In [ ]:
df_tor_mer.loc[df_tor_mer['Cluster Labels'] == 4, df_tor_mer.columns[[1] + list(range(5, df_tor_mer.shape[1]))]]

We have successfully cluster Toronto neighbourhood based on venue categories!